In [2]:
import h5py
import numpy as np

# Load test data from test_raw.h5
with h5py.File("../Data/test_psd.h5", "r") as f_test:
    X_test = f_test['data'][:]
    Y_test = f_test['labels'][:]
    S_test = f_test['sessions'][:]
    #H_test = f_test['hardwares'][:]  # Load the 'hardwares' dataset as h_test


# Now, X_combined, Y_combined, S_combined, and h_combined contain the merged data


# Now, X_combined, Y_combined, and S_combined contain the merged data


# Shuffle the data
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)

# Reorder the arrays according to the shuffled indices
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]
#h_test = H_test[indices]


# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []
h_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []
h_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    
    # Skip subjects with fewer than two unique sessions
    if len(np.unique(subject_sessions)) < 2:
        continue

    print(f"Subject {subject}, Number of unique sessions: {len(np.unique(subject_sessions))}")
    
    # Assign the minimum session to the evaluation set (x_test_e)
    min_session = np.min(subject_sessions)
    
    # Append data to the evaluation set (min session)
    x_test_e_list.extend(x_test[subject_indices][subject_sessions == min_session])
    y_test_e_list.extend(y_test[subject_indices][subject_sessions == min_session])
    s_test_e_list.extend(s_test[subject_indices][subject_sessions == min_session])
    #h_test_e_list.extend(h_test[subject_indices][subject_sessions == min_session])

    # Append remaining sessions to the validation set (x_test_v)
    x_test_v_list.extend(x_test[subject_indices][subject_sessions != min_session])
    y_test_v_list.extend(y_test[subject_indices][subject_sessions != min_session])
    s_test_v_list.extend(s_test[subject_indices][subject_sessions != min_session])
    #h_test_v_list.extend(h_test[subject_indices][subject_sessions != min_session])

# Shuffle and convert lists back to numpy arrays for x_test_e
indices_e = np.arange(len(x_test_e_list))
np.random.shuffle(indices_e)

x_test_e = np.array(x_test_e_list)[indices_e]
y_test_e = np.array(y_test_e_list)[indices_e]
s_test_e = np.array(s_test_e_list)[indices_e]
#h_test_e = np.array(h_test_v_list)[indices_e]

# Shuffle and convert lists back to numpy arrays for x_test_v
indices_v = np.arange(len(x_test_v_list))
np.random.shuffle(indices_v)

x_test_v = np.array(x_test_v_list)[indices_v]
y_test_v = np.array(y_test_v_list)[indices_v]
s_test_v = np.array(s_test_v_list)[indices_v]
#h_test_v = np.array(h_test_v_list)[indices_v]
# Optional: Save the new test evaluation and validation sets to npy files (if needed)


Subject 70, Number of unique sessions: 19
Subject 86, Number of unique sessions: 19
Subject 96, Number of unique sessions: 18
Subject 125, Number of unique sessions: 19
Subject 129, Number of unique sessions: 5
Subject 167, Number of unique sessions: 2
Subject 174, Number of unique sessions: 19
Subject 175, Number of unique sessions: 6
Subject 183, Number of unique sessions: 6
Subject 194, Number of unique sessions: 19
Subject 197, Number of unique sessions: 19
Subject 199, Number of unique sessions: 19
Subject 201, Number of unique sessions: 19
Subject 203, Number of unique sessions: 6
Subject 206, Number of unique sessions: 6
Subject 207, Number of unique sessions: 41
Subject 214, Number of unique sessions: 18
Subject 219, Number of unique sessions: 5
Subject 230, Number of unique sessions: 6
Subject 236, Number of unique sessions: 42
Subject 245, Number of unique sessions: 6
Subject 250, Number of unique sessions: 41
Subject 257, Number of unique sessions: 6
Subject 260, Number of u

In [3]:
with h5py.File("../Data/neg_psd.h5", "r") as f:
    x_neg_r = f['data'][:]
    y_neg = f['labels'][:]
    s_neg = f['sessions'][:]

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import cosine_similarity as cs


def EERf(resutls):
    resutls= np.array(resutls)
    genuine  = resutls[resutls[:, 1] == 1][:, 0]
    impostor = resutls[resutls[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return(stats_a.eer,stats_a.fmr100)

def calculate_and_print_averages(y_train, resutls3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    ii = 0

    for i in resutls3.keys():
        re = EERf(resutls3[i])
        eer = re[0]
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, Count = {counts[ii]}")
        eer_values.append(eer)
        ii += 1

    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")

import torch.nn.functional as F


def calculate_similarity_scores_two(enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance):
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    print(verification_embeddings.shape, enrollment_embeddings.shape)
    if distance == "cd":
        similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    elif distance == "ed":
        similarity_matrix = -1 * ed(verification_embeddings, enrollment_embeddings)

    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        predicted_scores = similarity_matrix[i]
        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]

        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            max_score = sum(sorted(predicted_scores[same_class_indices], reverse=True)[:10]) / 10
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls, i, cls])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls, i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two(enrollment_data, ye, verification_data, yv, e_network, distance):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two(
        enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance
    )
    return similarity_results_by_class, similarity_results_by_class_dict

def compute_embedding_batch_two(x_test_batch, embedding_network=None):
    print(x_test_batch.shape)
    return x_test_batch

In [7]:
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "ed")
calculate_and_print_averages(y_test_e, resutls3)

(5000, 11997)
(80643, 11997)
(80643, 11997) (5000, 11997)
70: EER = 0.5206, FMR100 = 0.9617, Count = 100
86: EER = 0.4744, FMR100 = 0.9817, Count = 100
96: EER = 0.4423, FMR100 = 0.9718, Count = 100
125: EER = 0.2856, FMR100 = 0.8439, Count = 100


/home/fallahi/anaconda3/envs/NeuroShield/lib/python3.9/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


129: EER = 0.6057, FMR100 = 0.9950, Count = 100
167: EER = 0.6726, FMR100 = 0.9900, Count = 100
174: EER = 0.5690, FMR100 = 0.9694, Count = 100
175: EER = 0.8220, FMR100 = 0.8220, Count = 100
183: EER = 0.4180, FMR100 = 0.9800, Count = 100
194: EER = 0.5791, FMR100 = 0.9072, Count = 100
197: EER = 0.4994, FMR100 = 0.9950, Count = 100
199: EER = 0.4311, FMR100 = 0.9944, Count = 100
201: EER = 0.6102, FMR100 = 0.9939, Count = 100
203: EER = 0.4565, FMR100 = 0.8760, Count = 100
206: EER = 0.8824, FMR100 = 0.9980, Count = 100
207: EER = 0.4765, FMR100 = 0.9955, Count = 100
214: EER = 0.4057, FMR100 = 0.9446, Count = 100
219: EER = 0.5800, FMR100 = 1.0000, Count = 100
230: EER = 0.5428, FMR100 = 0.9400, Count = 100
236: EER = 0.6027, FMR100 = 0.9878, Count = 100
245: EER = 0.5840, FMR100 = 0.9980, Count = 100
250: EER = 0.5628, FMR100 = 0.9915, Count = 100
257: EER = 0.3911, FMR100 = 0.9960, Count = 100
260: EER = 0.4996, FMR100 = 0.9669, Count = 100
261: EER = 0.4889, FMR100 = 0.9989, Coun

In [8]:
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "cd")
calculate_and_print_averages(y_test_e, resutls3)

(5000, 11997)
(80643, 11997)
(80643, 11997) (5000, 11997)


/home/fallahi/anaconda3/envs/NeuroShield/lib/python3.9/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


70: EER = 0.5336, FMR100 = 0.9822, Count = 100
86: EER = 0.3000, FMR100 = 0.8383, Count = 100
96: EER = 0.4982, FMR100 = 0.9894, Count = 100
125: EER = 0.3744, FMR100 = 0.8328, Count = 100
129: EER = 0.5936, FMR100 = 0.9775, Count = 100
167: EER = 0.3714, FMR100 = 1.0000, Count = 100
174: EER = 0.4644, FMR100 = 0.9278, Count = 100
175: EER = 0.2162, FMR100 = 0.5120, Count = 100
183: EER = 0.3549, FMR100 = 0.8560, Count = 100
194: EER = 0.4761, FMR100 = 0.8617, Count = 100
197: EER = 0.4300, FMR100 = 0.9356, Count = 100
199: EER = 0.3517, FMR100 = 0.9211, Count = 100
201: EER = 0.4789, FMR100 = 0.8139, Count = 100
203: EER = 0.2580, FMR100 = 0.8040, Count = 100
206: EER = 0.2921, FMR100 = 0.7680, Count = 100
207: EER = 0.4720, FMR100 = 0.9690, Count = 100
214: EER = 0.3830, FMR100 = 0.9625, Count = 100
219: EER = 0.6530, FMR100 = 1.0000, Count = 100
230: EER = 0.5720, FMR100 = 0.9840, Count = 100
236: EER = 0.5751, FMR100 = 0.9766, Count = 100
245: EER = 0.3880, FMR100 = 0.9120, Count =

In [6]:
# Flatten each array from 3D to 2D
x_test_e = x_test_e.reshape(x_test_e.shape[0], -1)
x_test_v= x_test_v.reshape(x_test_v.shape[0], -1)
x_test_n = x_neg_r.reshape(x_neg_r.shape[0], -1)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def calculate_eer(y_true, y_scores):
    # Separate genuine and impostor scores
    genuine_scores = y_scores[y_true == 1]
    impostor_scores = y_scores[y_true == 0]
    #print(y_true, y_scores)

    #print(genuine_scores, impostor_scores)
    print("genuine_scores: ", len(genuine_scores), sum(genuine_scores)/len(genuine_scores), "impostor_scores", len(impostor_scores), sum(impostor_scores)/len(impostor_scores))
    
    # Use pyeer to compute EER stats
    stats = get_eer_stats(genuine_scores, impostor_scores)
    
    return stats.eer

def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)

        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=20))
        ])
        
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        pipeline.fit(X, y)
        probabilities = pipeline.predict_proba(test_embeddings)[:, 1]

        # Calculate EER
        rf_eer = calculate_eer(test_labels, probabilities)

        print(f"Class {cls}: RandomForest EER = {rf_eer:.4f}")

        results[cls] = {
            'RandomForest': {'EER': rf_eer}
        }

    return results

def calculate_mean_std(results):

    # Extract EER values for each model
    model_eer_values = {}
    for cls, metrics in results.items():
        for model, model_metrics in metrics.items():
            if model not in model_eer_values:
                model_eer_values[model] = []
            model_eer_values[model].append(model_metrics['EER'])
    
    # Calculate mean and std
    model_stats = {}
    for model, eer_values in model_eer_values.items():
        mean = np.mean(eer_values) * 100
        std = np.std(eer_values) * 100
        model_stats[model] = {'mean': mean, 'std': std}
        print(f"Final Average EER {model}: {mean:.4f}")
        print(f"Final EER Standard Deviation {model}: {std:.4f}")
        print(f"${mean:.2f} \\pm {std:.2f}$") 
        
   
    return model_stats


In [8]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 11997)
(80643, 11997)
(84408, 11997)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.01873888888888892 impostor_scores 78843 0.0005056885202237429
Class 70: RandomForest EER = 0.3725
genuine_scores:  1800 0.054505555555555696 impostor_scores 78843 0.0006928960085232621
Class 86: RandomForest EER = 0.2039
genuine_scores:  1700 0.0006000000000000004 impostor_scores 78943 0.00010868601395943799
Class 96: RandomForest EER = 0.4791
genuine_scores:  1800 0.018944444444444496 impostor_scores 78843 0.000290957979782613
Class 125: RandomForest EER = 0.3792
genuine_scores:  400 0.011574999999999985 impostor_scores 80243 0.0006923968445845723
Class 129: RandomForest EER = 0.3590
genuine_scores:  100 0.0045000000000000005 impostor_scores 80543 0.0005844083284705066
Class 167: RandomForest EER = 0.

/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  4800 0.008785416666666698 impostor_scores 75843 0.0003085320992049433
Class 260: RandomForest EER = 0.3537
genuine_scores:  900 0.00465555555555554 impostor_scores 79743 0.0014290909546920028
Class 261: RandomForest EER = 0.4420
genuine_scores:  500 0.2663200000000001 impostor_scores 80143 0.0008936525959846857
Class 262: RandomForest EER = 0.0131
genuine_scores:  3900 0.029538461538461555 impostor_scores 76743 0.0009488813311963323
Class 265: RandomForest EER = 0.4668
genuine_scores:  1800 0.02425000000000001 impostor_scores 78843 0.0005476706873152974
Class 268: RandomForest EER = 0.3556
genuine_scores:  1800 0.04283333333333334 impostor_scores 78843 0.00040346004084066035
Class 270: RandomForest EER = 0.4759
genuine_scores:  1800 0.0004333333333333336 impostor_scores 78843 0.000638103572923404
Class 281: RandomForest EER = 0.4939
genuine_scores:  1800 0.011522222222222215 impostor_scores 78843 0.0005427241479903107
Class 291: RandomForest EER = 0.4175
genuine_scores

/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  2100 0.004085714285714268 impostor_scores 78543 0.0003302649504093391
Class 299: RandomForest EER = 0.4356
genuine_scores:  2000 0.04112000000000018 impostor_scores 78643 0.00031814656104167574
Class 303: RandomForest EER = 0.2075
genuine_scores:  2200 0.014872727272727282 impostor_scores 78443 6.33581071606134e-05
Class 306: RandomForest EER = 0.4453
genuine_scores:  500 0.0009400000000000005 impostor_scores 80143 0.0002791260621638868
Class 314: RandomForest EER = 0.4667
genuine_scores:  2000 0.06135000000000026 impostor_scores 78643 0.000532151621886248
Class 322: RandomForest EER = 0.2854
genuine_scores:  2100 0.03653809523809541 impostor_scores 78543 0.0003529276956571674
Class 325: RandomForest EER = 0.1463
genuine_scores:  2200 0.041795454545454754 impostor_scores 78443 0.0005713702943538638
Class 342: RandomForest EER = 0.1322
genuine_scores:  2200 0.0013318181818181795 impostor_scores 78443 0.0006372780235330111
Class 348: RandomForest EER = 0.4721
genuine_sco

{'RandomForest': {'mean': 32.48705220226814, 'std': 14.034232167306678}}

In [9]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    """
    Learn classifiers per unique subject and evaluate using EER.
    """
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('kernel_approximation', RBFSampler(gamma=1, random_state=42, n_components=500)),  # Approximate RBF kernel
            ('classifier', SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, class_weight='balanced', random_state=42, n_jobs=20))
        ])
        
        # Shuffle and split the data
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        
        # Train the pipeline
        pipeline.fit(X, y)
        
        # Predict probabilities for the test set
        # Note: SGDClassifier doesn't provide probabilities directly, so we use decision_function
        decision_scores = pipeline.decision_function(test_embeddings)
        probabilities = 1 / (1 + np.exp(-decision_scores))  # Sigmoid function to map decision scores to probabilities
        
        # Calculate EER
        svm_eer = calculate_eer(test_labels, probabilities)
        
        print(f"Class {cls}: Linear SVM with Kernel Approximation EER = {svm_eer:.4f}")
        
        results[cls] = {
            'RandomForest': {'EER': svm_eer}
        }
        
    return results



In [10]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 11997)
(80643, 11997)
(84408, 11997)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.11153378261075743 impostor_scores 78843 0.03154267780440607
Class 70: Linear SVM with Kernel Approximation EER = 0.3636
genuine_scores:  1800 0.24301692265542618 impostor_scores 78843 0.045161860787646765
Class 86: Linear SVM with Kernel Approximation EER = 0.3397
genuine_scores:  1700 0.12764587537629316 impostor_scores 78943 0.07860396698175744
Class 96: Linear SVM with Kernel Approximation EER = 0.3865
genuine_scores:  1800 0.07003236370765904 impostor_scores 78843 0.05212357809301892
Class 125: Linear SVM with Kernel Approximation EER = 0.5252
genuine_scores:  400 0.23336896127612097 impostor_scores 80243 0.15665051635348412
Class 129: Linear SVM with Kernel Approximation EER = 0.4150
genuine_score

/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  1800 0.1820342764980338 impostor_scores 78843 0.048542047214340765
Class 268: Linear SVM with Kernel Approximation EER = 0.3889
genuine_scores:  1800 0.12872828904959568 impostor_scores 78843 0.0986619063508477
Class 270: Linear SVM with Kernel Approximation EER = 0.4128
genuine_scores:  1800 0.16644512218214733 impostor_scores 78843 0.04672485536656158
Class 281: Linear SVM with Kernel Approximation EER = 0.3501
genuine_scores:  1800 0.20933898781877824 impostor_scores 78843 0.09257846490694976
Class 291: Linear SVM with Kernel Approximation EER = 0.4758
genuine_scores:  2200 0.08656290908008779 impostor_scores 78443 0.08324385386823968
Class 296: Linear SVM with Kernel Approximation EER = 0.5400


/home/fallahi/anaconda3/envs/oml/lib/python3.9/site-packages/pyeer/eer_stats.py:219: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  2100 0.16246712134354258 impostor_scores 78543 0.03422356230037985
Class 299: Linear SVM with Kernel Approximation EER = 0.3667
genuine_scores:  2000 0.2595824447117084 impostor_scores 78643 0.11885446208554715
Class 303: Linear SVM with Kernel Approximation EER = 0.3262
genuine_scores:  2200 0.22070992032023715 impostor_scores 78443 0.09276557405857622
Class 306: Linear SVM with Kernel Approximation EER = 0.4473
genuine_scores:  500 0.25724630214304606 impostor_scores 80143 0.08133491185411543
Class 314: Linear SVM with Kernel Approximation EER = 0.3421
genuine_scores:  2000 0.24261316850354717 impostor_scores 78643 0.13006197220132634
Class 322: Linear SVM with Kernel Approximation EER = 0.3551
genuine_scores:  2100 0.5251921700751262 impostor_scores 78543 0.2480098075742251
Class 325: Linear SVM with Kernel Approximation EER = 0.3121
genuine_scores:  2200 0.7154879797545357 impostor_scores 78443 0.09560884270896171
Class 342: Linear SVM with Kernel Approximation EER

{'RandomForest': {'mean': 32.977301480452205, 'std': 12.162724522917433}}